In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle 
import plotly.express as px


from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split


In [ ]:
from sklearn.cluster import KMeans

In [ ]:
df = pd.read_csv("../db/fixed_smartphones.csv")

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
#Необходимо вырезать ссылку на продукт.
#Выбрать основные кампании 
#Избавиться от имени
#Подрезать знак рубля в цене
#Подрезать покупки
#Перевести процентовку рекомендуют в флоат
#Количество обзоров в интов
#Избавиться от мегапикселей
#Избавиться от мА ч
#Избавиться от " в диагонали
#Тыкнуть в ROM только первую часть и избавиться от гб
#Избавиться от гб в РАМ
#Онехотнуть OS
#Кластеризировать прикол
#

In [ ]:
df['OS'].unique()

In [ ]:
company_names = ['Blackview','Ulefone','KingKong','Xiaomi','Cubot','Oscal','Blackview','Ulefone','Oukitel','Google','Apple']

In [ ]:
bf_cleaned = df.dropna()

In [ ]:
bf_cleaned.info()

In [ ]:
bf_cleaned.head()

In [ ]:
bf_cleaned = bf_cleaned.reset_index()

In [ ]:
bf_cleaned.info()

In [ ]:
bf_cleaned = bf_cleaned.drop(['index', 'link'],axis=1)

In [ ]:
for i in bf_cleaned['name']:
    print(i)

In [ ]:
for index, row in bf_cleaned.iterrows():
    for i in company_names:
        if i in str(row['name']):
            bf_cleaned.at[index, 'name'] = i

In [ ]:
for index, row in bf_cleaned.iterrows():
    word = row['price'].replace('₽', ' ')
    word = word.replace('\xa0', '')
    bf_cleaned.at[index, 'price'] = int(word)

In [ ]:
for index, row in bf_cleaned.iterrows():
    word = row['sales'].split(' ')[0]
    if word == 'более':
        word = row['sales'].split(' ')[1]
    bf_cleaned.at[index, 'sales'] = int(word)

In [ ]:
for index, row in bf_cleaned.iterrows():
    word = row['stars'].split(',')
    try:
        bf_cleaned.at[index, 'stars'] = float(word[0]+'.'+word[1])
    except:
        bf_cleaned.at[index, 'stars'] = float(word[0]+'.'+'0')

In [ ]:
for index, row in bf_cleaned.iterrows():
    word = row['rec_proc'].split('%')[0]
    bf_cleaned.at[index, 'rec_proc'] = float(word[0])/10

In [ ]:
for index, row in bf_cleaned.iterrows():
    word = row['RAM'].split(' ')[0]
    bf_cleaned.at[index, 'RAM'] = int(word)

In [ ]:
for index, row in bf_cleaned.iterrows():
    word = row['ROM'].split(' ')[0]
    bf_cleaned.at[index, 'ROM'] = int(word)

In [ ]:
for index, row in bf_cleaned.iterrows():
    word = row['diag'].replace('"', '')
    word = word.replace(',', '.')
    bf_cleaned.at[index, 'diag'] = float(word)

In [ ]:
for index, row in bf_cleaned.iterrows():
    word = row['bat'].split(' ')[0]
    word = word.replace('\xa0', '')
    bf_cleaned.at[index, 'bat'] = int(word)

In [ ]:
for index, row in bf_cleaned.iterrows():
    word = row['cam'].split(' ')[0]
    bf_cleaned.at[index, 'cam'] = int(word)

In [ ]:
bf_cleaned.head()

In [ ]:
bf_cleaned.info() 

In [ ]:
bf_cleaned = bf_cleaned.astype({'price': int,'sales': int,'review_num': int,
            'stars': float,'rec_proc': float,'RAM': int,
            'ROM': int,'diag': float,'bat': int,'cam': int,}) 


In [ ]:
final_df = bf_cleaned.copy()

In [ ]:
bf_cleaned['OS'].unique()

In [ ]:
categorical_cols = bf_cleaned.select_dtypes('object').columns.tolist()
encoder = OneHotEncoder(sparse_output=False,drop = None,
                        dtype = np.float64,
                        handle_unknown='ignore', sparse=False)

encoder.fit(bf_cleaned[categorical_cols])
encoded_cols = list(encoder.get_feature_names_out(categorical_cols))
bf_cleaned[encoded_cols] = encoder.transform(bf_cleaned[categorical_cols])


In [ ]:
bf_cleaned = bf_cleaned.drop(categorical_cols, axis=1)

In [ ]:
input_columns = ['price', 'sales', 'stars', 'rec_proc', 'review_num','RAM', 'ROM', 'diag', 'bat', 'cam']
scaler = MinMaxScaler()
scaler.fit(bf_cleaned[input_columns])
bf_cleaned[input_columns] = scaler.transform(bf_cleaned[input_columns])

In [ ]:
bf_cleaned.head()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_samples, silhouette_score

In [ ]:
for n_clusters in range(2,6):
    clusterer = KMeans(n_clusters=n_clusters, random_state=42,n_init="auto")
    cluster_labels = clusterer.fit_predict(bf_cleaned)
    silhouette_avg = silhouette_score(bf_cleaned, cluster_labels)
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg,
    )

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=42, n_init="auto").fit(bf_cleaned)
final_df["kmeans_4"] = kmeans.labels_

In [ ]:
final_df.head()

In [ ]:
import pickle
with open('../db/ready_data.pkl', 'wb') as f:
    pickle.dump(final_df, f)